In [ ]:
import pandas as pd
import numpy as np
import pathlib
import matplotlib.pyplot as plt
import sys
sys.path.append('ext')
import plot_util
import accelerometers

In [ ]:
inputdir = pathlib.Path.cwd() / 'inputdata'

In [ ]:
outputdir = pathlib.Path.cwd() / 'outputdata'
outputdir.mkdir(exist_ok=True)

In [ ]:
pathlib.Path.cwd()

In [ ]:
# Adapt paths if needed.
# I use two layers of sub-directories to link to an icloud directory with sub-directories named based on trip category.

files = list(inputdir.glob('*/*/Accelerometer.csv'))
{i: files[i] for i in np.arange(len(files))}

In [ ]:
ps_collection = {} # Save for subsequent plots
for file in files:
    if len(files) > 5:
        suppress_figures = True
    else:
        suppress_figures = False
    df = accelerometers.clean_file(file)

    title = str(file.parents[1].stem) + '-- ' + str(file.parent.stem)
    print('\n', title)
    
    # Norm over time, in units of g-force
    plt.figure()
    plot_util.generic_plot(accelerometers.smooth(df[['g-force']], 100), kind='line', xlabel='Sec Elapsed', ylabel='g',
                           title=title + '_gforce', output_directory=outputdir)
    ps = accelerometers.ps(df, smooth_window=100)
    if suppress_figures:
        plt.close()
        
    # Save for later
    ps_collection[title] = ps.copy()

    # Log power spectrum
    plt.figure()
    plot_util.generic_plot(np.log10(ps[accelerometers.POWER_LABEL]), kind='line', xlabel='Hz', ylabel='log10(W/kg/Hz)',
                           title=title, output_directory=outputdir)
    if suppress_figures:
        plt.close()


    # Log power spectrum
    plt.figure()
    plot_util.generic_plot(np.log10(ps[accelerometers.POWER_LABEL]).rename(np.log10, axis=0), kind='line', xlabel='log10(Hz)', ylabel='log10(W/kg/Hz)',
                           title=title + '_logx', output_directory=outputdir)
    if suppress_figures:
        plt.close()
    
    total_pow = np.trapz(ps[accelerometers.POWER_LABEL], x=ps.index)
    print('Total Power: ' + str(int(total_pow)) + ' W/kg')
    print('Mean g-force: ' + str(df['g-force'].mean().round(2)) + " g's")


In [ ]:
len(ps_collection)

In [ ]:
combined = accelerometers.interp_combine(ps_collection)

In [ ]:
combined.to_csv(outputdir / 'Combined.csv')

In [ ]:
combined_bycat = combined.groupby('Category', axis=1).mean()
plot_util.generic_plot(np.log10(combined_bycat).rename(np.log10, axis=0), kind='line', xlabel='log10(Hz)', ylabel='log10(W/kg/Hz)',
                       output_directory=outputdir, title='Average Power Spectra by Mode')

Everything from here down will need to be customized.

In [ ]:
# Make combined plot of selected
selected_idx = [17, 10, 14, 8, 6, 4, 1]#, 2]
{i:files[i] for i in selected_idx}

In [ ]:
# Define common frequency range and interp
freq = np.logspace(-1, 1.5, 500)
small_combined = pd.DataFrame(index=freq)
labels = ['Biking, stable phone pos.', 'Stationary Control', 'Biking, phone in pocket',
         'BART, Rough', 'BART, Smoother', 'Driving', 'Walking, stable phone pos.']#, 'Walking, phone in pocket']
for i in range(len(selected_idx)):
    ps = list(ps_collection.values())[selected_idx[i]]
    small_combined[labels[i]] = np.interp(x=small_combined.index, xp=ps.index, fp=ps[accelerometers.POWER_LABEL])
small_combined.head()

In [ ]:
plot_util.generic_plot(np.log10(small_combined).rename(np.log10, axis=0), kind='line', xlabel='log10(Hz)', ylabel='log10(W/kg/Hz)',
                       output_directory=outputdir, title='Selected Power Spectra')

In [ ]:
excerpt = ['Walking, stable phone pos.', 'Driving', 'BART, Rough', 'BART, Smoother']
plot_util.generic_plot(np.log10(small_combined[excerpt]).rename(np.log10, axis=0), kind='line', xlabel='log10(Hz)', ylabel='log10(W/kg/Hz)',
                       output_directory=outputdir, title='Excerpted Power Spectra')